# Аналитика полётной активности гражданских беспилотников в РФ для АНО "Федеральный центр беспилотных авиационных систем" 

# 1. Загрузка данных 

In [3]:
import pandas as pd
import re
from datetime import datetime, timedelta
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon

Загрузим датасет

In [5]:
df = pd.read_excel(PATH)

Загрузим границы регионов

In [7]:
gdf_regions = gpd.read_file(PATH)

# 2. Предобработка данных

Напишем функцию для парсинга необходимых полей из файла (стандартизированные сообщения в Госкорпорацию ОРВД)

In [10]:
def parse_shr_table(df):
    parsed = []

    for _, row in df.iterrows():
        shr = str(row["SHR"])
        dep_text = str(row.get("DEP", ""))
        arr_text = str(row.get("ARR", ""))
        center = row["Центр ЕС ОрВД"]

        data = {
            "Центр ЕС": center,
            "SID": None,
            "DOF": None,
            "TYP": None,
            "REG": None,
            "OPR": None,
            "DEP": None,
            "DEST": None,
            "DEP_TIME": None,
            "ARR_TIME": None,
            "Высота_от": None,
            "Высота_до": None,
        }

        # SID
        sid_match = re.search(r"SID/(\d+)", shr)
        if sid_match:
            data["SID"] = sid_match.group(1)

        # Дата операции
        dof_match = re.search(r"DOF/(\d{6})", shr)
        if dof_match:
            dof_raw = dof_match.group(1)
            data["DOF"] = datetime.strptime(dof_raw, "%y%m%d").date()

        # Тип БПЛА
        typ_match = re.search(r"TYP/([\w\d]+)", shr)
        if typ_match:
            data["TYP"] = typ_match.group(1)

        # Регистрационный номер
        reg_match = re.search(r"REG/([\w\d, ]+)", shr)
        if reg_match:
            data["REG"] = reg_match.group(1).strip()

        # Оператор
        opr_match = re.search(r"OPR/(.*?)(?=\s*(REG/|TYP/|DOF/|DEP/|DEST/|SID/|ORGN/|RMK/|$))",shr,flags=re.DOTALL)
        if opr_match:
            data["OPR"] = opr_match.group(1).replace("\n", " ").strip()


        # DEP и DEST (обычный способ)
        dep_match = re.search(r"DEP/([A-Z0-9]+)", shr)
        dest_match = re.search(r"DEST/([A-Z0-9]+)", shr)
        if dep_match:
            data["DEP"] = dep_match.group(1)
        if dest_match:
            data["DEST"] = dest_match.group(1)

        # Если DEP/DEST не нашли, то ищем координаты в SHR
        if not data["DEP"] or not data["DEST"]:
            coords = re.findall(r"\d{6}[NS]\d{7}[EW]|\d{4}[NS]\d{5}[EW]", shr)
            if coords:
                if not data["DEP"]:
                    data["DEP"] = coords[0]
                if len(coords) > 1 and not data["DEST"]:
                    data["DEST"] = coords[1]

        # Если все еще пусто, то ищем в DEP/ARR в ADEPZ/ADARRZ
        if not data["DEP"]:
            depz_match = re.search(r"ADEPZ\s+(\d+[NS]\d+[EW])", dep_text)
            if depz_match:
                data["DEP"] = depz_match.group(1)

        if not data["DEST"]:
            arrz_match = re.search(r"ADARRZ\s+(\d+[NS]\d+[EW])", arr_text)
            if arrz_match:
                data["DEST"] = arrz_match.group(1)

        # Высота
        alt_match = re.search(r"M(\d{4})/M(\d{4})", shr)
        if alt_match:
            data["Высота_от"] = int(alt_match.group(1))
            data["Высота_до"] = int(alt_match.group(2))

        # Время вылета
        dep_time_match = re.search(r"ATD\s*(\d{4})", dep_text)
        if dep_time_match:
            data["DEP_TIME"] = f"{dep_time_match.group(1)[:2]}:{dep_time_match.group(1)[2:]}"
        else:
            shr_dep_time = re.search(r"-\w{4}(\d{4})", shr)
            if shr_dep_time:
                dep_raw = shr_dep_time.group(1)
                data["DEP_TIME"] = f"{dep_raw[:2]}:{dep_raw[2:]}"

        # Время прилета
        arr_time_match = re.search(r"ATA\s*(\d{4})", arr_text)
        if arr_time_match:
            data["ARR_TIME"] = f"{arr_time_match.group(1)[:2]}:{arr_time_match.group(1)[2:]}"
        else:
            data["ARR_TIME"] = pd.NA

        parsed.append(data)

    return pd.DataFrame(parsed)

Применим функцию

In [12]:
df_clean = parse_shr_table(df)

In [13]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76902 entries, 0 to 76901
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Центр ЕС   76902 non-null  object 
 1   SID        76902 non-null  object 
 2   DOF        76902 non-null  object 
 3   TYP        76902 non-null  object 
 4   REG        60782 non-null  object 
 5   OPR        75908 non-null  object 
 6   DEP        76502 non-null  object 
 7   DEST       76397 non-null  object 
 8   DEP_TIME   76902 non-null  object 
 9   ARR_TIME   74324 non-null  object 
 10  Высота_от  72600 non-null  float64
 11  Высота_до  72600 non-null  float64
dtypes: float64(2), object(10)
memory usage: 7.0+ MB


Проверим на дубликаты

In [15]:
df_clean.duplicated().sum()

0

Напишем несколько функций
- для преобразования координат вида '5957N02905E' в (59.950000, 29.083333)
- для получения максимальной высоты
- для получения максимального время полета

In [17]:
# Конвертация координат
def convert_coord(coord_str):
    if pd.isna(coord_str) or not isinstance(coord_str, str):
        return None, None
    try:
        # формат с секундами
        match = re.match(r"(\d{2})(\d{2})(\d{2})([NS])(\d{3})(\d{2})(\d{2})([EW])", coord_str)
        if match:
            d1, m1, s1, ns, d2, m2, s2, ew = match.groups()
            lat = int(d1) + int(m1)/60 + int(s1)/3600
            lon = int(d2) + int(m2)/60 + int(s2)/3600
            if ns == "S": lat = -lat
            if ew == "W": lon = -lon
            return round(lat, 6), round(lon, 6)
        # формат без секунд
        match = re.match(r"(\d{2})(\d{2})([NS])(\d{3})(\d{2})([EW])", coord_str)
        if match:
            d1, m1, ns, d2, m2, ew = match.groups()
            lat = int(d1) + int(m1)/60
            lon = int(d2) + int(m2)/60
            if ns == "S": lat = -lat
            if ew == "W": lon = -lon
            return round(lat, 6), round(lon, 6)
    except:
        return None, None
    return None, None

# Время полета
def calc_flight_time(dep, arr):
    if pd.isna(dep) or pd.isna(arr):
        return None
    try:
        dep_dt = datetime.strptime(dep, "%H:%M")
        arr_dt = datetime.strptime(arr, "%H:%M")
        if arr_dt < dep_dt:  # через полночь
            arr_dt += timedelta(days=1)
        return int((arr_dt - dep_dt).total_seconds() / 60)
    except:
        return None

def enrich_df(df):
    # координаты DEP
    coords_dep = df["DEP"].apply(convert_coord)
    df["lat_dep"] = coords_dep.apply(lambda x: x[0] if x else None)
    df["lon_dep"] = coords_dep.apply(lambda x: x[1] if x else None)

    # координаты DEST
    coords_dest = df["DEST"].apply(convert_coord)
    df["lat_dest"] = coords_dest.apply(lambda x: x[0] if x else None)
    df["lon_dest"] = coords_dest.apply(lambda x: x[1] if x else None)

    # макс высота
    df["Высота_макс"] = df[["Высота_от", "Высота_до"]].max(axis=1)

    # время полета
    df["Время_полета_мин"] = df.apply(
        lambda row: calc_flight_time(row["DEP_TIME"], row["ARR_TIME"]), axis=1
    )

    return df

Применим функцию

In [19]:
df = enrich_df(df_clean)

Напишем функцию для извлечения числа бпла из столбца с типами

In [21]:
def get_number(typ_value):
        if pd.isna(typ_value) or not isinstance(typ_value, str):
            return 1
        
        # Ищем последовательность цифр в строке
        numbers = re.findall(r'\d+', typ_value)
        
        if numbers:
            # Берем первое найденное число
            return int(numbers[0])
        else:
            # Если чисел нет - возвращаем 1
            return 1

Создадим столбец с количеством БПЛА из столбца с типом

In [23]:
df['TYP_NUMBER'] = df['TYP'].apply(get_number)

Удалим все цифры из столбца с типом

In [25]:
df['TYP'] = df['TYP'].str.replace(r'^\d+', '', regex=True)

In [26]:
df['TYP'].unique()

array(['SHAR', 'BLA', 'AER'], dtype=object)

Посмотрим пропуски

In [28]:
df.isna().mean()

Центр ЕС            0.000000
SID                 0.000000
DOF                 0.000000
TYP                 0.000000
REG                 0.209617
OPR                 0.012926
DEP                 0.005201
DEST                0.006567
DEP_TIME            0.000000
ARR_TIME            0.033523
Высота_от           0.055941
Высота_до           0.055941
lat_dep             0.005201
lon_dep             0.005201
lat_dest            0.006567
lon_dest            0.006567
Высота_макс         0.055941
Время_полета_мин    0.033523
TYP_NUMBER          0.000000
dtype: float64

In [29]:
df['DOF'] = pd.to_datetime(df['DOF'])

Создаём GeoDataFrame с точками вылета БПЛА (столбец geometry), чтобы связать их с границами регионов из `gdf_regions`

In [31]:
gdf_points = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon_dep"], df["lat_dep"]),
    crs="EPSG:4326" 
)

In [32]:
print(gdf_regions.crs)

EPSG:4326


In [33]:
print(gdf_points.crs)

EPSG:4326


Объединим геодатафреймы

In [35]:
gdf_joined = gpd.sjoin(gdf_points, gdf_regions, how="left", predicate="within")

# Убираем дубли (если точка попала в несколько регионов)
gdf_joined = gdf_joined.drop_duplicates(subset=["SID"]) 

Преобразуем обратно в обычный датафрейм

In [37]:
df_normal = pd.DataFrame(gdf_joined.drop(columns='geometry'))

Оставим необходимые столбцы, остальные удалим

In [39]:
df_new = df_normal[['Центр ЕС', 'SID', 'DOF', 'OPR', 'TYP', 'REG', 'DEP', 'DEST', 'DEP_TIME', 'ARR_TIME', 'Высота_от', 'Высота_до', 'lat_dep', 'lon_dep', 'lat_dest', 'lon_dest', 'Высота_макс', 'Время_полета_мин', 'TYP_NUMBER', 'name']].copy()

Добавим дополнительные столбцы

In [41]:
df_new['month'] = df_new['DOF'].dt.month

In [42]:
df_new['day_of_week'] = df_new['DOF'].dt.day_name()

In [43]:
df_new['DEP_TIME_dt'] = pd.to_datetime(df_new['DEP_TIME'], format='%H:%M', errors='coerce')

In [44]:
df_new['ARR_TIME_dt'] = pd.to_datetime(df_new['ARR_TIME'], format='%H:%M', errors='coerce')

In [45]:
df_new['hour'] = df_new['DEP_TIME_dt'].dt.hour

In [46]:
df_new['day_type'] = df_new['day_of_week'].map(lambda d: 'Выходной' if d in ['Saturday', 'Sunday'] else 'Будни')